# 신용카드

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np

# Selenium 설정 및 드라이버 실행
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)  # 브라우저 꺼짐 방지
chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])  # 불필요한 에러 메시지 없애기
driver = webdriver.Chrome(options=chrome_options)

# 카드 링크 리스트 초기화
credit_card = []

try:
    driver.get('https://card-gorilla.com/search/card?cate=CRD')

    # 더보기 버튼을 더 이상 누를 수 없을 때까지 클릭
    while True:
        try:
            more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="q-app"]/section/div[1]/section/div/article[1]/article/a'))
            )
            driver.execute_script("arguments[0].click();", more_button)  # JavaScript로 클릭
            time.sleep(2)  # 클릭 후 잠시 대기
        except Exception:
            print("더보기 버튼을 더 이상 클릭할 수 없습니다.")
            break  # 더 이상 버튼을 클릭할 수 없으면 반복 종료

    # 모든 카드 링크 가져오기
    card_elements = driver.find_elements(By.CSS_SELECTOR, 'div.card_data > div.benefit > a.b_view')
    for index, card in enumerate(card_elements):
        href = card.get_attribute('href')
        credit_card.append(href)

    print("총 수집된 카드 링크 수:", len(credit_card))

except Exception as e:
    print(f"카드 링크를 가져오는 데 실패했습니다: {e}")

finally:
    driver.quit()


더보기 버튼을 더 이상 클릭할 수 없습니다.
총 수집된 카드 링크 수: 975


In [ ]:
# 수집한 카드 링크를 DataFrame으로 변환
credit_df = pd.DataFrame(credit_card, columns=['Card_Link'])

# CSV 파일로 저장
credit_df.to_csv('credit_card_links.csv', index=False, encoding='utf-8-sig')  # 한글 포함 시 'utf-8-sig' 권장

print("CSV 파일로 저장되었습니다: credit_card_links.csv") 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time

credit_card = pd.read_csv('credit_card_links.csv', encoding='utf-8-sig')


# Selenium 설정 및 드라이버 실행
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)  # 브라우저 꺼짐 방지
chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])  # 불필요한 에러 메세지 없애기
driver = webdriver.Chrome(options=chrome_options)


# 키워드 리스트
keywords = ["편의점", "마트/편의점", #편의점
            "카페", "카페/디저트", "베이커리", #카페/디저트 
            "일반음식점", "점심", "푸드", "패스트푸드", #외식
            "주유", "주유소", #주유
            "영화", "영화/문화", #영화/문화
            "대형마트", "마트/편의점", #마트
            "쇼핑", "온라인쇼핑", "백화점", #쇼핑
            "병원/약국", "병원", #병원/약국
            "교육/육아", "학원", #교육/육아
            "통신", "SKT", #통신
            "자동차", "자동차/하이패스", "하이패스",  #자동차/하이패스
            "여행/숙박", "여행", #여행/숙박
            "대중교통", "교통"] #대중교통

# 결과를 저장할 데이터프레임 초기화
columns = ['id', 'name', 'brand', 'img_url', 'card_url',
           'convenience', 'cafe', 'restaurant', 'oil', 'movie', 'mart', 'shopping', 'hospital',
           'edu', 'tel', 'car', 'travel', 'transportation']
card_data = []

for i in range(len(credit_card)):
    link = list(credit_card.iloc[i])[0]
    # 링크에서 id 추출
    card_id = link.split('/')[-1]
    card_info = {
        'id': card_id, 
        'name': None, 
        'brand': None, 
        'img_url': None, 
        'card_url': link,
        'cafe': None, 
        'restaurant': None, 
        'oil': None, 
        'movie': None, 
        'mart': None, 
        'shopping': None, 
        'hospital': None,
        'edu': None, 
        'tel': None, 
        'car': None, 
        'travel': None, 
        'transportation': None
    }
    try:
        # 각 카드 상세 페이지로 이동
        driver.get(link)
        time.sleep(2)  # 페이지 로딩 대기

        # 페이지의 헤더 가리기 (선택 사항)
        driver.execute_script('document.querySelector("#q-app > header").style.visibility="hidden";')

        # 카드 이름 가져오기
        card_name_element = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.tit > strong')
        card_info['name'] = card_name_element.text

        # 카드 브랜드 가져오기
        card_brand_element = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.tit > p')
        card_info['brand'] = card_brand_element.text

        #카드 이미지 가져오기
        try:
            card_image_element = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.plate_area > div > img')
        except:
            # 이미지 요소가 로드될 때까지 대기
            card_image_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'img[data-v-6a7d0b5e]')) #카드 이미지가 여러 개인 경우 제일 앞에 노출된 카드 이미지 수집
            )
        finally:
            # WebDriver 종료
            card_info['img_url'] = card_image_element.get_attribute('src')

        # 혜택 정보 추출
        dl_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.lst.bene_area > dl'))
        )

        for index, dl in enumerate(dl_elements):
            try:
                # dt 요소 내의 p.txt1 요소의 텍스트 확인
                dt_element = dl.find_element(By.TAG_NAME, 'dt')
                p_txt1 = dt_element.find_element(By.CSS_SELECTOR, 'p.txt1')

                if any(keyword in p_txt1.text for keyword in keywords):
                    # 스크롤하여 해당 토글 버튼이 화면에 보이게 하기
                    driver.execute_script("arguments[0].scrollIntoView(true);", dt_element)
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(dt_element))
                    time.sleep(2)  # 스크롤 후 안정화 대기

                    # 토글 클릭
                    ActionChains(driver).move_to_element(dt_element).click(dt_element).perform()

                    # <dd> 요소나 div.in_box > p가 나타날 때까지 대기
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'dd, div.in_box > p'))
                    )

                    # <dd> 요소가 있는지 확인 후 내용 추출
                    dd_elements = dl.find_elements(By.TAG_NAME, 'dd')
                    benefit_text = ""
                    if dd_elements:
                        benefit_text = dd_elements[0].find_element(By.CLASS_NAME, 'in_box').text
                    else:
                        # <dd> 요소가 없는 경우 div.in_box > p로 내용 가져오기
                        p_elements = dl.find_elements(By.CSS_SELECTOR, 'div.in_box > p')
                        benefit_text = " ".join([p.text.replace("\n", " ") for p in p_elements])

                    # 혜택 정보를 적절한 컬럼에 저장
                    if p_txt1.text in ["편의점", "마트/편의점"]:
                        card_info['convenience'] = benefit_text
                    elif p_txt1.text in ["카페", "카페/디저트", "베이커리"]:
                        card_info['cafe'] = benefit_text
                    elif p_txt1.text in ["일반음식점", "점심", "푸드", "패스트푸드"]:
                        card_info['restaurant'] = benefit_text
                    elif p_txt1.text in ["주유", "주유소"]:
                        card_info['oil'] = benefit_text
                    elif p_txt1.text in ["영화", "영화/문화"]:
                        card_info['movie'] = benefit_text
                    elif p_txt1.text in ["대형마트", "마트/편의점"]:
                        card_info['mart'] = benefit_text
                    elif p_txt1.text in ["쇼핑", "온라인쇼핑", "백화점"]:
                        card_info['shopping'] = benefit_text
                    elif p_txt1.text in ["병원/약국", "병원"]:
                        card_info['hospital'] = benefit_text
                    elif p_txt1.text in ["교육/육아", "학원"]:
                        card_info['edu'] = benefit_text
                    elif p_txt1.text in ["통신", "SKT"]:
                        card_info['tel'] = benefit_text
                    elif p_txt1.text in ["자동차", "자동차/하이패스", "하이패스"]:
                        card_info['car'] = benefit_text
                    elif p_txt1.text in ["여행/숙박", "여행"]:
                        card_info['travel'] = benefit_text
                    elif p_txt1.text in ["대중교통", "교통"]:
                        card_info['transportation'] = benefit_text

            except Exception as e:
                print(f"토글 {index + 1} 내용을 가져오는 데 실패했습니다: {e}")

        # 수집한 카드 정보 추가
        card_data.append(card_info)

    except Exception as e:
        print(f"카드 {link}의 정보를 가져오는 데 실패했습니다: {e}")

driver.quit()

# DataFrame 생성
df = pd.DataFrame(card_data, columns=columns)
print(df)

# 체크카드

In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np

# Selenium 설정 및 드라이버 실행
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)  # 브라우저 꺼짐 방지
chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])  # 불필요한 에러 메시지 없애기s
driver = webdriver.Chrome(options=chrome_options)

# 카드 링크 리스트 초기화
check_card = []

try:
    driver.get('https://card-gorilla.com/search/card?cate=CHK')

    # 더보기 버튼을 더 이상 누를 수 없을 때까지 클릭
    while True:
        try:
            more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="q-app"]/section/div[1]/section/div/article[1]/article/a'))
            )
            driver.execute_script("arguments[0].click();", more_button)  # JavaScript로 클릭
            time.sleep(2)  # 클릭 후 잠시 대기
        except Exception:
            print("더보기 버튼을 더 이상 클릭할 수 없습니다.")
            break  # 더 이상 버튼을 클릭할 수 없으면 반복 종료

    # 모든 카드 링크 가져오기
    card_elements = driver.find_elements(By.CSS_SELECTOR, 'div.card_data > div.benefit > a.b_view')
    for index, card in enumerate(card_elements):
        href = card.get_attribute('href')
        check_card.append(href)

    print("총 수집된 카드 링크 수:", len(check_card))

except Exception as e:
    print(f"카드 링크를 가져오는 데 실패했습니다: {e}")

finally:
    driver.quit()


더보기 버튼을 더 이상 클릭할 수 없습니다.
총 수집된 카드 링크 수: 423


In [39]:
# 수집한 카드 링크를 DataFrame으로 변환
check_df = pd.DataFrame(check_card, columns=['Card_Link'])

# CSV 파일로 저장
check_df.to_csv('check_card_links.csv', index=False, encoding='utf-8-sig')  # 한글 포함 시 'utf-8-sig' 권장

print("CSV 파일로 저장되었습니다: check_card_links.csv") 

CSV 파일로 저장되었습니다: check_card_links.csv


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time

check_card = pd.read_csv('check_card_links.csv', encoding='utf-8-sig')


# Selenium 설정 및 드라이버 실행
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)  # 브라우저 꺼짐 방지
chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])  # 불필요한 에러 메세지 없애기
driver = webdriver.Chrome(options=chrome_options)


# 키워드 리스트
keywords = ["편의점", "마트/편의점", #편의점
            "카페", "카페/디저트", "베이커리", #카페/디저트 
            "일반음식점", "점심", "푸드", "패스트푸드", #외식
            "주유", "주유소", #주유
            "영화", "영화/문화", #영화/문화
            "대형마트", "마트/편의점", #마트
            "쇼핑", "온라인쇼핑", "백화점", #쇼핑
            "병원/약국", "병원", #병원/약국
            "교육/육아", "학원", #교육/육아
            "통신", "SKT", #통신
            "자동차", "자동차/하이패스", "하이패스",  #자동차/하이패스
            "여행/숙박", "여행", #여행/숙박
            "대중교통", "교통"] #대중교통

# 결과를 저장할 데이터프레임 초기화
columns = ['id', 'name', 'brand', 'img_url', 'card_url',
           'convenience', 'cafe', 'restaurant', 'oil', 'movie', 'mart', 'shopping', 'hospital',
           'edu', 'tel', 'car', 'travel', 'transportation']
check_card_data = []

for i in range(len(check_card)):
    link = list(check_card.iloc[i])[0]
    # 링크에서 id 추출
    card_id = link.split('/')[-1]
    check_card_info = {
        'id': card_id, 
        'name': None, 
        'brand': None, 
        'img_url': None, 
        'card_url': link,
        'cafe': None, 
        'restaurant': None, 
        'oil': None, 
        'movie': None, 
        'mart': None, 
        'shopping': None, 
        'hospital': None,
        'edu': None, 
        'tel': None, 
        'car': None, 
        'travel': None, 
        'transportation': None
    }
    try:
        # 각 카드 상세 페이지로 이동
        driver.get(link)
        time.sleep(2)  # 페이지 로딩 대기

        # 페이지의 헤더 가리기 (선택 사항)
        driver.execute_script('document.querySelector("#q-app > header").style.visibility="hidden";')

        # 카드 이름 가져오기
        card_name_element = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.tit > strong')
        check_card_info['name'] = card_name_element.text

        # 카드 브랜드 가져오기
        card_brand_element = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.tit > p')
        check_card_info['brand'] = card_brand_element.text

        #카드 이미지 가져오기
        try:
            card_image_element = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.plate_area > div > img')
        except:
            # 이미지 요소가 로드될 때까지 대기
            card_image_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'img[data-v-6a7d0b5e]')) #카드 이미지가 여러 개인 경우 제일 앞에 노출된 카드 이미지 수집
            )
        finally:
            # WebDriver 종료
            check_card_info['img_url'] = card_image_element.get_attribute('src')

        # 혜택 정보 추출
        dl_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.lst.bene_area > dl'))
        )

        for index, dl in enumerate(dl_elements):
            try:
                # dt 요소 내의 p.txt1 요소의 텍스트 확인
                dt_element = dl.find_element(By.TAG_NAME, 'dt')
                p_txt1 = dt_element.find_element(By.CSS_SELECTOR, 'p.txt1')

                if any(keyword in p_txt1.text for keyword in keywords):
                    # 스크롤하여 해당 토글 버튼이 화면에 보이게 하기
                    driver.execute_script("arguments[0].scrollIntoView(true);", dt_element)
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(dt_element))
                    time.sleep(2)  # 스크롤 후 안정화 대기

                    # 토글 클릭
                    ActionChains(driver).move_to_element(dt_element).click(dt_element).perform()

                    # <dd> 요소나 div.in_box > p가 나타날 때까지 대기
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'dd, div.in_box > p'))
                    )

                    # <dd> 요소가 있는지 확인 후 내용 추출
                    dd_elements = dl.find_elements(By.TAG_NAME, 'dd')
                    benefit_text = ""
                    if dd_elements:
                        benefit_text = dd_elements[0].find_element(By.CLASS_NAME, 'in_box').text
                    else:
                        # <dd> 요소가 없는 경우 div.in_box > p로 내용 가져오기
                        p_elements = dl.find_elements(By.CSS_SELECTOR, 'div.in_box > p')
                        benefit_text = " ".join([p.text.replace("\n", " ") for p in p_elements])

                    # 혜택 정보를 적절한 컬럼에 저장
                    if p_txt1.text in ["편의점", "마트/편의점"]:
                        check_card_info['convenience'] = benefit_text
                    elif p_txt1.text in ["카페", "카페/디저트", "베이커리"]:
                        check_card_info['cafe'] = benefit_text
                    elif p_txt1.text in ["일반음식점", "점심", "푸드", "패스트푸드"]:
                        check_card_info['restaurant'] = benefit_text
                    elif p_txt1.text in ["주유", "주유소"]:
                        check_card_info['oil'] = benefit_text
                    elif p_txt1.text in ["영화", "영화/문화"]:
                        check_card_info['movie'] = benefit_text
                    elif p_txt1.text in ["대형마트", "마트/편의점"]:
                        check_card_info['mart'] = benefit_text
                    elif p_txt1.text in ["쇼핑", "온라인쇼핑", "백화점"]:
                        check_card_info['shopping'] = benefit_text
                    elif p_txt1.text in ["병원/약국", "병원"]:
                        check_card_info['hospital'] = benefit_text
                    elif p_txt1.text in ["교육/육아", "학원"]:
                        check_card_info['edu'] = benefit_text
                    elif p_txt1.text in ["통신", "SKT"]:
                        check_card_info['tel'] = benefit_text
                    elif p_txt1.text in ["자동차", "자동차/하이패스", "하이패스"]:
                        check_card_info['car'] = benefit_text
                    elif p_txt1.text in ["여행/숙박", "여행"]:
                        check_card_info['travel'] = benefit_text
                    elif p_txt1.text in ["대중교통", "교통"]:
                        check_card_info['transportation'] = benefit_text

            except Exception as e:
                print(f"토글 {index + 1} 내용을 가져오는 데 실패했습니다: {e}")

        # 수집한 카드 정보 추가
        check_card_data.append(check_card_info)

    except Exception as e:
        print(f"카드 {link}의 정보를 가져오는 데 실패했습니다: {e}")

driver.quit()

# DataFrame 생성
check_df = pd.DataFrame(check_card_data, columns=columns)
print(check_df)

/Users/juran/Desktop/github/fisa_final/fisa/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


토글 5 내용을 가져오는 데 실패했습니다: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=130.0.6723.117)
Stacktrace:
0   chromedriver                        0x00000001027c3648 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x00000001027bbea8 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x0000000102228104 cxxbridge1$string$len + 88416
3   chromedriver                        0x0000000102213704 cxxbridge1$string$len + 3936
4   chromedriver                        0x000000010221348c cxxbridge1$string$len + 3304
5   chromedriver                        0x000000010221183c core::str::slice_error_fail::h1cab30ac4b13c655 + 60316
6   chromedriver                        0x0000000102211dd0 core::str::slice_error_fail::h1cab30ac4b13c655 + 61744
7   chromedriver                        0x0000000102230018 cxxbridge1$string$len + 120948
8   chromedriver    

In [4]:
check_df.to_csv('check1.csv')

In [5]:
check_df.tail(3)

,id,name,brand,img_url,card_url,convenience,cafe,restaurant,oil,movie,mart,shopping,hospital,edu,tel,car,travel,transportation
99,370,BAZIC(베이직)체크카드,NH농협카드,https://d1c5n4ri2guedi.cloudfront.net/card/370...,https://card-gorilla.com/card/detail/370,NaN,None,None,None,None,None,None,None,None,None,None,None,None
100,393,삼성체크카드 & POINT,삼성카드,https://d1c5n4ri2guedi.cloudfront.net/card/393...,https://card-gorilla.com/card/detail/393,NaN,None,None,None,"CGV 온라인예매 3,000원 캐시백\n- CGV 홈페이지 및 스마트폰 App을 통...",None,None,None,None,None,None,None,None
101,2454,트리플카드,한패스,https://d1c5n4ri2guedi.cloudfront.net/card/245...,https://card-gorilla.com/card/detail/2454,NaN,None,None,None,None,None,None,None,None,None,None,None,None


In [8]:
check_card2 = check_card[102:]

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time

check_card = pd.read_csv('check_card_links.csv', encoding='utf-8-sig')


# Selenium 설정 및 드라이버 실행
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)  # 브라우저 꺼짐 방지
chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])  # 불필요한 에러 메세지 없애기
driver = webdriver.Chrome(options=chrome_options)


# 키워드 리스트
keywords = ["편의점", "마트/편의점", #편의점
            "카페", "카페/디저트", "베이커리", #카페/디저트 
            "일반음식점", "점심", "푸드", "패스트푸드", #외식
            "주유", "주유소", #주유
            "영화", "영화/문화", #영화/문화
            "대형마트", "마트/편의점", #마트
            "쇼핑", "온라인쇼핑", "백화점", #쇼핑
            "병원/약국", "병원", #병원/약국
            "교육/육아", "학원", #교육/육아
            "통신", "SKT", #통신
            "자동차", "자동차/하이패스", "하이패스",  #자동차/하이패스
            "여행/숙박", "여행", #여행/숙박
            "대중교통", "교통"] #대중교통

# 결과를 저장할 데이터프레임 초기화
columns = ['id', 'name', 'brand', 'img_url', 'card_url',
           'convenience', 'cafe', 'restaurant', 'oil', 'movie', 'mart', 'shopping', 'hospital',
           'edu', 'tel', 'car', 'travel', 'transportation']
check_card_data = []

for i in range(len(check_card2)):
    link = list(check_card2.iloc[i])[0]
    # 링크에서 id 추출
    card_id = link.split('/')[-1]
    check_card_info = {
        'id': card_id, 
        'name': None, 
        'brand': None, 
        'img_url': None, 
        'card_url': link,
        'cafe': None, 
        'restaurant': None, 
        'oil': None, 
        'movie': None, 
        'mart': None, 
        'shopping': None, 
        'hospital': None,
        'edu': None, 
        'tel': None, 
        'car': None, 
        'travel': None, 
        'transportation': None
    }
    try:
        # 각 카드 상세 페이지로 이동
        driver.get(link)
        time.sleep(2)  # 페이지 로딩 대기

        # 페이지의 헤더 가리기 (선택 사항)
        driver.execute_script('document.querySelector("#q-app > header").style.visibility="hidden";')

        # 카드 이름 가져오기
        card_name_element = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.tit > strong')
        check_card_info['name'] = card_name_element.text

        # 카드 브랜드 가져오기
        card_brand_element = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.tit > p')
        check_card_info['brand'] = card_brand_element.text

        #카드 이미지 가져오기
        try:
            card_image_element = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.plate_area > div > img')
        except:
            # 이미지 요소가 로드될 때까지 대기
            card_image_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'img[data-v-6a7d0b5e]')) #카드 이미지가 여러 개인 경우 제일 앞에 노출된 카드 이미지 수집
            )
        finally:
            # WebDriver 종료
            check_card_info['img_url'] = card_image_element.get_attribute('src')

        # 혜택 정보 추출
        dl_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.lst.bene_area > dl'))
        )

        for index, dl in enumerate(dl_elements):
            try:
                # dt 요소 내의 p.txt1 요소의 텍스트 확인
                dt_element = dl.find_element(By.TAG_NAME, 'dt')
                p_txt1 = dt_element.find_element(By.CSS_SELECTOR, 'p.txt1')

                if any(keyword in p_txt1.text for keyword in keywords):
                    # 스크롤하여 해당 토글 버튼이 화면에 보이게 하기
                    driver.execute_script("arguments[0].scrollIntoView(true);", dt_element)
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(dt_element))
                    time.sleep(2)  # 스크롤 후 안정화 대기

                    # 토글 클릭
                    ActionChains(driver).move_to_element(dt_element).click(dt_element).perform()

                    # <dd> 요소나 div.in_box > p가 나타날 때까지 대기
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'dd, div.in_box > p'))
                    )

                    # <dd> 요소가 있는지 확인 후 내용 추출
                    dd_elements = dl.find_elements(By.TAG_NAME, 'dd')
                    benefit_text = ""
                    if dd_elements:
                        benefit_text = dd_elements[0].find_element(By.CLASS_NAME, 'in_box').text
                    else:
                        # <dd> 요소가 없는 경우 div.in_box > p로 내용 가져오기
                        p_elements = dl.find_elements(By.CSS_SELECTOR, 'div.in_box > p')
                        benefit_text = " ".join([p.text.replace("\n", " ") for p in p_elements])

                    # 혜택 정보를 적절한 컬럼에 저장
                    if p_txt1.text in ["편의점", "마트/편의점"]:
                        check_card_info['convenience'] = benefit_text
                    elif p_txt1.text in ["카페", "카페/디저트", "베이커리"]:
                        check_card_info['cafe'] = benefit_text
                    elif p_txt1.text in ["일반음식점", "점심", "푸드", "패스트푸드"]:
                        check_card_info['restaurant'] = benefit_text
                    elif p_txt1.text in ["주유", "주유소"]:
                        check_card_info['oil'] = benefit_text
                    elif p_txt1.text in ["영화", "영화/문화"]:
                        check_card_info['movie'] = benefit_text
                    elif p_txt1.text in ["대형마트", "마트/편의점"]:
                        check_card_info['mart'] = benefit_text
                    elif p_txt1.text in ["쇼핑", "온라인쇼핑", "백화점"]:
                        check_card_info['shopping'] = benefit_text
                    elif p_txt1.text in ["병원/약국", "병원"]:
                        check_card_info['hospital'] = benefit_text
                    elif p_txt1.text in ["교육/육아", "학원"]:
                        check_card_info['edu'] = benefit_text
                    elif p_txt1.text in ["통신", "SKT"]:
                        check_card_info['tel'] = benefit_text
                    elif p_txt1.text in ["자동차", "자동차/하이패스", "하이패스"]:
                        check_card_info['car'] = benefit_text
                    elif p_txt1.text in ["여행/숙박", "여행"]:
                        check_card_info['travel'] = benefit_text
                    elif p_txt1.text in ["대중교통", "교통"]:
                        check_card_info['transportation'] = benefit_text

            except Exception as e:
                print(f"토글 {index + 1} 내용을 가져오는 데 실패했습니다: {e}")

        # 수집한 카드 정보 추가
        check_card_data.append(check_card_info)

    except Exception as e:
        print(f"카드 {link}의 정보를 가져오는 데 실패했습니다: {e}")

driver.quit()

# DataFrame 생성
check_df = pd.DataFrame(check_card_data, columns=columns)
print(check_df)

카드 https://card-gorilla.com/card/detail/2096의 정보를 가져오는 데 실패했습니다: Message: javascript error: Cannot read properties of null (reading 'style')
  (Session info: chrome=130.0.6723.117)
Stacktrace:
0   chromedriver                        0x0000000103027648 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x000000010301fea8 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x0000000102a8c104 cxxbridge1$string$len + 88416
3   chromedriver                        0x0000000102a9119c cxxbridge1$string$len + 109048
4   chromedriver                        0x0000000102a92b0c cxxbridge1$string$len + 115560
5   chromedriver                        0x0000000102b08808 cxxbridge1$string$len + 598116
6   chromedriver                        0x0000000102b07bd0 cxxbridge1$string$len + 594988
7   chromedriver                        0x0000000102ac2f54 cxxbridge1$string$len + 313264
8   chromedriver                        0x0000000102ac3ba4 cxxbridge1$string$len + 316416
9 

In [10]:
check_df.to_csv('check2.csv')